In [31]:
import os
import time
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_recall_fscore_support
from sklearn.metrics import f1_score,roc_auc_score
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

df = pd.read_csv(r"UNSW-NB15.csv")

In [32]:
label_names = df['attack_cat'].unique()

In [33]:
features = ['dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss',
 'sintpkt', 'dintpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat', 'smeansz', 'dmeansz', 'trans_depth',
 'res_bdy_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'is_ftp_login',
 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm', 'ct_srv_dst', 'is_sm_ips_ports', 'attack_cat']

print(features)

['dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss', 'sintpkt', 'dintpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat', 'smeansz', 'dmeansz', 'trans_depth', 'res_bdy_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm', 'ct_srv_dst', 'is_sm_ips_ports', 'attack_cat']


In [34]:
df = df[features]

In [35]:
df

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,sttl,dttl,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat
0,0.001055,udp,dns,CON,2,2,132,164,31,29,...,1,1,1,0.0,0.0,0.0,3,7,0,Normal
1,0.036133,udp,-,CON,4,4,528,304,31,29,...,1,1,2,0.0,0.0,0.0,3,4,0,Normal
2,0.001119,udp,dns,CON,2,2,146,178,31,29,...,2,1,1,0.0,0.0,0.0,2,8,0,Normal
3,0.001209,udp,dns,CON,2,2,132,164,31,29,...,1,1,1,0.0,0.0,0.0,1,9,0,Normal
4,0.001169,udp,dns,CON,2,2,146,178,31,29,...,1,1,1,0.0,0.0,0.0,1,9,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2539734,0.087306,tcp,ftp-data,FIN,6,8,320,1828,31,29,...,1,1,3,-1.0,-1.0,-1.0,3,2,0,Normal
2539735,0.365058,tcp,ftp,CON,8,6,456,346,31,29,...,2,2,2,2.0,2.0,-1.0,2,2,0,Normal
2539736,6.335154,tcp,ftp,CON,32,30,1802,2088,31,29,...,2,2,2,2.0,2.0,-1.0,2,2,0,Normal
2539737,2.200934,tcp,http,CON,58,116,3498,166054,31,29,...,2,2,2,-1.0,-1.0,2.0,4,1,0,Normal


In [36]:
labelencoder = LabelEncoder()
for col_name in df.dtypes[df.dtypes == 'object'].index:
      df[col_name] = labelencoder.fit_transform(df[col_name])

In [37]:
df

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,sttl,dttl,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat
0,0.001055,119,2,2,2,2,132,164,31,29,...,1,1,1,0.0,0.0,0.0,3,7,0,6
1,0.036133,119,0,2,4,4,528,304,31,29,...,1,1,2,0.0,0.0,0.0,3,4,0,6
2,0.001119,119,2,2,2,2,146,178,31,29,...,2,1,1,0.0,0.0,0.0,2,8,0,6
3,0.001209,119,2,2,2,2,132,164,31,29,...,1,1,1,0.0,0.0,0.0,1,9,0,6
4,0.001169,119,2,2,2,2,146,178,31,29,...,1,1,1,0.0,0.0,0.0,1,9,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2539734,0.087306,113,4,5,6,8,320,1828,31,29,...,1,1,3,-1.0,-1.0,-1.0,3,2,0,6
2539735,0.365058,113,3,2,8,6,456,346,31,29,...,2,2,2,2.0,2.0,-1.0,2,2,0,6
2539736,6.335154,113,3,2,32,30,1802,2088,31,29,...,2,2,2,2.0,2.0,-1.0,2,2,0,6
2539737,2.200934,113,5,2,58,116,3498,166054,31,29,...,2,2,2,-1.0,-1.0,2.0,4,1,0,6


In [38]:
# import pandas as pd
# from imblearn.over_sampling import SMOTE
# from imblearn.under_sampling import RandomUnderSampler

# # Load the dataset
# # df = pd.read_csv('UNSW-NB15.csv')

# # Inspect the dataset
# print(df.head())
# print(df['attack_cat'].value_counts())  # Replace 'target' with your actual target column name

# # Separate features and target
# X = df.drop('attack_cat', axis=1)  # Replace 'target' with your actual target column name
# y = df['attack_cat']

# # Apply SMOTE for oversampling
# smote = SMOTE(random_state=42)
# X_res, y_res = smote.fit_resample(X, y)
# df_res = pd.DataFrame(X_res, columns=X.columns)
# df_res['attack_cat'] = y_res
# df_res.to_csv('oversampled_dataset.csv', index=False)
# print("After SMOTE oversampling:")
# print(df_res['attack_cat'].value_counts())

# # Apply RandomUnderSampler for undersampling
# undersample = RandomUnderSampler(random_state=42)
# X_res, y_res = undersample.fit_resample(X, y)
# df_res = pd.DataFrame(X_res, columns=X.columns)
# df_res['attack_cat'] = y_res
# df_res.to_csv('undersampled_dataset.csv', index=False)
# print("After Random undersampling:")
# print(df_res['attack_cat'].value_counts())


In [39]:
X = df.drop('attack_cat', axis=1)  # Replace 'target' with your actual target column name
y = df['attack_cat']

In [40]:
from collections import Counter
class_counts = Counter(y)
sampling_strategy = {class_label: int(0.10 * max(class_counts.values())) for class_label in class_counts}

In [41]:
class_counts

Counter({6: 2218456,
         5: 215481,
         3: 44525,
         4: 24246,
         2: 16353,
         7: 13987,
         0: 2677,
         1: 2329,
         8: 1511,
         9: 174})

In [42]:
sampling_strategy

{6: 221845,
 3: 221845,
 7: 221845,
 2: 221845,
 5: 221845,
 8: 221845,
 4: 221845,
 9: 221845,
 1: 221845,
 0: 221845}

In [43]:
sampling_strategy[6] = 2218456

In [44]:
df

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,sttl,dttl,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat
0,0.001055,119,2,2,2,2,132,164,31,29,...,1,1,1,0.0,0.0,0.0,3,7,0,6
1,0.036133,119,0,2,4,4,528,304,31,29,...,1,1,2,0.0,0.0,0.0,3,4,0,6
2,0.001119,119,2,2,2,2,146,178,31,29,...,2,1,1,0.0,0.0,0.0,2,8,0,6
3,0.001209,119,2,2,2,2,132,164,31,29,...,1,1,1,0.0,0.0,0.0,1,9,0,6
4,0.001169,119,2,2,2,2,146,178,31,29,...,1,1,1,0.0,0.0,0.0,1,9,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2539734,0.087306,113,4,5,6,8,320,1828,31,29,...,1,1,3,-1.0,-1.0,-1.0,3,2,0,6
2539735,0.365058,113,3,2,8,6,456,346,31,29,...,2,2,2,2.0,2.0,-1.0,2,2,0,6
2539736,6.335154,113,3,2,32,30,1802,2088,31,29,...,2,2,2,2.0,2.0,-1.0,2,2,0,6
2539737,2.200934,113,5,2,58,116,3498,166054,31,29,...,2,2,2,-1.0,-1.0,2.0,4,1,0,6


In [45]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load your dataset (replace with your data loading code)
# df = pd.read_csv('UNSW-NB15.csv')

# Suppose 'attack_cat', 'proto', 'service', 'state' are columns to exclude from features
exclude_columns = ['proto', 'service', 'state','attack_cat']

# Separate features and target
features = df.drop(exclude_columns, axis=1)
target_columns = ['proto', 'service', 'state','attack_cat']
target = df[target_columns]  # Assuming these are the target columns

# Normalize features (excluding target) with non-negative values
scaler = MinMaxScaler(feature_range=(0, 1))  # Set range to 0-1 for non-negative values
scaled_features = scaler.fit_transform(features)
features_scaled = pd.DataFrame(scaled_features, columns=features.columns)

# Combine scaled features with the original target columns
df = pd.concat([features_scaled, target], axis=1)

print(df)


                  dur     spkts     dpkts    sbytes    dbytes      sttl  \
0        1.200687e-07  0.000188  0.000182  0.000009  0.000011  0.121569   
1        4.112267e-06  0.000376  0.000363  0.000037  0.000021  0.121569   
2        1.273525e-07  0.000188  0.000182  0.000010  0.000012  0.121569   
3        1.375953e-07  0.000188  0.000182  0.000009  0.000011  0.121569   
4        1.330429e-07  0.000188  0.000182  0.000010  0.000012  0.121569   
...               ...       ...       ...       ...       ...       ...   
2539734  9.936224e-06  0.000564  0.000726  0.000022  0.000125  0.121569   
2539735  4.154695e-05  0.000751  0.000545  0.000032  0.000024  0.121569   
2539736  7.209987e-04  0.003006  0.002723  0.000126  0.000142  0.121569   
2539737  2.504865e-04  0.005448  0.010528  0.000244  0.011329  0.121569   
2539738  1.073202e-04  0.001127  0.001089  0.000040  0.000046  0.243137   

             dttl         sload     dload     sloss  ...  is_ftp_login  \
0        0.114173  8.3579

In [46]:
df

,dur,spkts,dpkts,sbytes,dbytes,sttl,dttl,sload,dload,sloss,...,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,proto,service,state,attack_cat
0,1.200687e-07,0.000188,0.000182,0.000009,0.000011,0.121569,0.114173,8.357948e-05,0.004829,0.000000,...,0.2,0.111111,0.027027,0.030303,0.090909,0.0,119,2,2,6
1,4.112267e-06,0.000376,0.000363,0.000037,0.000021,0.121569,0.114173,1.464196e-05,0.000392,0.000000,...,0.2,0.111111,0.027027,0.030303,0.045455,0.0,119,0,2,6
2,1.273525e-07,0.000188,0.000182,0.000010,0.000012,0.121569,0.114173,8.715673e-05,0.004942,0.000000,...,0.2,0.111111,0.027027,0.015152,0.106061,0.0,119,2,2,6
3,1.375953e-07,0.000188,0.000182,0.000009,0.000011,0.121569,0.114173,7.293329e-05,0.004214,0.000000,...,0.2,0.111111,0.027027,0.000000,0.121212,0.0,119,2,2,6
4,1.330429e-07,0.000188,0.000182,0.000010,0.000012,0.121569,0.114173,8.342890e-05,0.004730,0.000000,...,0.2,0.111111,0.027027,0.000000,0.121212,0.0,119,2,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2539734,9.936224e-06,0.000564,0.000726,0.000022,0.000125,0.121569,0.114173,4.085783e-06,0.001139,0.000188,...,0.0,0.000000,0.000000,0.030303,0.015152,0.0,113,4,5,6
2539735,4.154695e-05,0.000751,0.000545,0.000032,0.000024,0.121569,0.114173,1.460223e-06,0.000049,0.000376,...,0.6,0.333333,0.000000,0.015152,0.015152,0.0,113,3,2,6
2539736,7.209987e-04,0.003006,0.002723,0.000126,0.000142,0.121569,0.114173,3.682097e-07,0.000020,0.001316,...,0.6,0.333333,0.000000,0.015152,0.015152,0.0,113,3,2,6
2539737,2.504865e-04,0.005448,0.010528,0.000244,0.011329,0.121569,0.114173,2.086926e-06,0.004647,0.000376,...,0.0,0.000000,0.081081,0.045455,0.000000,0.0,113,5,2,6


In [47]:
print(df['attack_cat'].unique())

[6 3 7 2 5 8 4 9 1 0]


In [48]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# Load the dataset
# df = pd.read_csv('UNSW-NB15.csv')

# Inspect the dataset
print(df.head())
print(df['attack_cat'].value_counts())  # Replace 'target' with your actual target column name

# Separate features and target
X = df.drop('attack_cat', axis=1)  # Replace 'target' with your actual target column name
y = df['attack_cat']
# Apply SMOTE for oversampling
smote = SMOTE(sampling_strategy=sampling_strategy,random_state=42)
X_res , y_res = smote.fit_resample(X ,y)
df_res = pd.DataFrame(X_res, columns=X.columns)
df_res['attack_cat'] = y_res
df_res.to_csv('oversampled_dataset.csv', index=False)
print("After SMOTE oversampling:")
print(df_res['attack_cat'].value_counts())

            dur     spkts     dpkts    sbytes    dbytes      sttl      dttl  \
0  1.200687e-07  0.000188  0.000182  0.000009  0.000011  0.121569  0.114173   
1  4.112267e-06  0.000376  0.000363  0.000037  0.000021  0.121569  0.114173   
2  1.273525e-07  0.000188  0.000182  0.000010  0.000012  0.121569  0.114173   
3  1.375953e-07  0.000188  0.000182  0.000009  0.000011  0.121569  0.114173   
4  1.330429e-07  0.000188  0.000182  0.000010  0.000012  0.121569  0.114173   

      sload     dload  sloss  ...  is_ftp_login  ct_ftp_cmd  ct_flw_http_mthd  \
0  0.000084  0.004829    0.0  ...           0.2    0.111111          0.027027   
1  0.000015  0.000392    0.0  ...           0.2    0.111111          0.027027   
2  0.000087  0.004942    0.0  ...           0.2    0.111111          0.027027   
3  0.000073  0.004214    0.0  ...           0.2    0.111111          0.027027   
4  0.000083  0.004730    0.0  ...           0.2    0.111111          0.027027   

   ct_src_ltm  ct_srv_dst  is_sm_ips_p

In [49]:
import os
import time
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_recall_fscore_support
from sklearn.metrics import f1_score,roc_auc_score
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

df = pd.read_csv(r"oversampled_dataset.csv")

In [50]:
df

,dur,spkts,dpkts,sbytes,dbytes,sttl,dttl,sload,dload,sloss,...,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,proto,service,state,attack_cat
0,1.200687e-07,0.000188,0.000182,0.000009,0.000011,0.121569,0.114173,8.357948e-05,0.004829,0.000000,...,0.2,0.111111,0.027027,0.030303,0.090909,0.0,119,2,2,6
1,4.112267e-06,0.000376,0.000363,0.000037,0.000021,0.121569,0.114173,1.464196e-05,0.000392,0.000000,...,0.2,0.111111,0.027027,0.030303,0.045455,0.0,119,0,2,6
2,1.273525e-07,0.000188,0.000182,0.000010,0.000012,0.121569,0.114173,8.715673e-05,0.004942,0.000000,...,0.2,0.111111,0.027027,0.015152,0.106061,0.0,119,2,2,6
3,1.375953e-07,0.000188,0.000182,0.000009,0.000011,0.121569,0.114173,7.293329e-05,0.004214,0.000000,...,0.2,0.111111,0.027027,0.000000,0.121212,0.0,119,2,2,6
4,1.330429e-07,0.000188,0.000182,0.000010,0.000012,0.121569,0.114173,8.342890e-05,0.004730,0.000000,...,0.2,0.111111,0.027027,0.000000,0.121212,0.0,119,2,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4215056,6.401341e-04,0.005979,0.031994,0.000213,0.031485,0.996078,0.992126,7.152315e-07,0.004839,0.000376,...,0.0,0.000000,0.054054,0.000000,0.000000,0.0,113,5,5,9
4215057,6.349819e-10,0.000188,0.000000,0.000143,0.000000,0.996078,0.000000,2.474353e-01,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.006373,0.0,119,0,6,9
4215058,1.018040e-04,0.000939,0.000545,0.000091,0.000018,0.996078,0.992126,1.923820e-06,0.000017,0.000376,...,0.0,0.000000,0.054054,0.000000,0.000000,0.0,113,5,5,9
4215059,4.107566e-04,0.004361,0.021366,0.000165,0.021317,0.996078,0.992126,8.615791e-07,0.005356,0.000376,...,0.0,0.000000,0.001282,0.000719,0.000000,0.0,113,5,5,9


In [51]:
y = df["attack_cat"]

In [52]:
df = df.drop("attack_cat",axis=1)

In [53]:
X = df 

In [54]:
from sklearn.model_selection import train_test_split

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [56]:
set(y_train)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [57]:
import gc
del X
del df
del y
del labelencoder
gc.collect()

0

In [58]:
from sklearn.metrics import classification_report, confusion_matrix

In [60]:
import tensorflow as tf
from tensorflow import keras
X_train = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)
# Define the model (same as before)
model = keras.Sequential([
  keras.Input(shape=(41,1)),
  keras.layers.LSTM(50, activation="relu",return_sequences=False),
  keras.layers.Dense(10, activation="softmax")
])

# Compile the model with optimizer, loss function, and metrics
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])
# Train the model (replace with your training data)


model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
92205/92205 ━━━━━━━━━━━━━━━━━━━━ 736s 8ms/step - accuracy: 0.7565 - loss: 0.6502 - val_accuracy: 0.8353 - val_loss: 0.4315
Epoch 2/10
92205/92205 ━━━━━━━━━━━━━━━━━━━━ 712s 8ms/step - accuracy: 0.8267 - loss: 0.4520 - val_accuracy: 0.8290 - val_loss: 0.4506
Epoch 3/10
92205/92205 ━━━━━━━━━━━━━━━━━━━━ 710s 8ms/step - accuracy: 0.8311 - loss: 0.4408 - val_accuracy: 0.8348 - val_loss: 0.4282
Epoch 4/10
92205/92205 ━━━━━━━━━━━━━━━━━━━━ 790s 9ms/step - accuracy: 0.8472 - loss: 125.4282 - val_accuracy: 0.8551 - val_loss: 0.3766
Epoch 5/10
92205/92205 ━━━━━━━━━━━━━━━━━━━━ 739s 8ms/step - accuracy: 0.8529 - loss: 0.3813 - val_accuracy: 0.8530 - val_loss: 0.3776
Epoch 6/10
92205/92205 ━━━━━━━━━━━━━━━━━━━━ 710s 8ms/step - accuracy: 0.8571 - loss: 0.3694 - val_accuracy: 0.8553 - val_loss: 0.3679
Epoch 7/10
92205/92205 ━━━━━━━━━━━━━━━━━━━━ 731s 8ms/step - accuracy: 0.8589 - loss: 0.3692 - val_accuracy: 0.8608 - val_loss: 0.3577
Epoch 8/10
92205/92205 ━━━━━━━━━━━━━━━━━━━━ 706s 8ms/step - 

In [29]:
from sklearn.tree import ExtraTreeClassifier, DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming X_train, X_test, y_train, y_test are already defined

def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    return accuracy, precision, recall, f1

# ExtraTreeClassifier
extra_tree_model = ExtraTreeClassifier()
accuracy, precision, recall, f1 = evaluate_model(extra_tree_model, X_train, y_train, X_test, y_test)
print("ExtraTreeClassifier:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# DecisionTreeClassifier
decision_tree_model = DecisionTreeClassifier()
accuracy, precision, recall, f1 = evaluate_model(decision_tree_model, X_train, y_train, X_test, y_test)
print("\nDecisionTreeClassifier:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# RandomForestClassifier
random_forest_model = RandomForestClassifier()
accuracy, precision, recall, f1 = evaluate_model(random_forest_model, X_train, y_train, X_test, y_test)
print("\nRandomForestClassifier:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# LogisticRegression
logistic_regression_model = LogisticRegression(max_iter=1000)
accuracy, precision, recall, f1 = evaluate_model(logistic_regression_model, X_train, y_train, X_test, y_test)
print("\nLogisticRegression:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# LinearDiscriminantAnalysis
lda_model = LinearDiscriminantAnalysis()
accuracy, precision, recall, f1 = evaluate_model(lda_model, X_train, y_train, X_test, y_test)
print("\nLinearDiscriminantAnalysis:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# QuadraticDiscriminantAnalysis
qda_model = QuadraticDiscriminantAnalysis()
accuracy, precision, recall, f1 = evaluate_model(qda_model, X_train, y_train, X_test, y_test)
print("\nQuadraticDiscriminantAnalysis:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# RidgeClassifier
ridge_model = RidgeClassifier()
accuracy, precision, recall, f1 = evaluate_model(ridge_model, X_train, y_train, X_test, y_test)
print("\nRidgeClassifier:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# AdaBoostClassifier
adaboost_model = AdaBoostClassifier()
accuracy, precision, recall, f1 = evaluate_model(adaboost_model, X_train, y_train, X_test, y_test)
print("\nAdaBoostClassifier:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# GradientBoostingClassifier
gradient_boosting_model = GradientBoostingClassifier()
accuracy, precision, recall, f1 = evaluate_model(gradient_boosting_model, X_train, y_train, X_test, y_test)
print("\nGradientBoostingClassifier:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# BaggingClassifier
bagging_model = BaggingClassifier()
accuracy, precision, recall, f1 = evaluate_model(bagging_model, X_train, y_train, X_test, y_test)
print("\nBaggingClassifier:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


ExtraTreeClassifier:
Accuracy: 0.9025305274179352
Precision: 0.9079633158407548
Recall: 0.9025305274179352
F1 Score: 0.9021033481794825

DecisionTreeClassifier:
Accuracy: 0.9070792926005857
Precision: 0.9125670998411434
Recall: 0.9070792926005857
F1 Score: 0.9067456395532058

RandomForestClassifier:
Accuracy: 0.9137964712273995
Precision: 0.9203992925254386
Recall: 0.9137964712273995
F1 Score: 0.9139836819960522


C:\Users\Dodooz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



LogisticRegression:
Accuracy: 0.7727713067182067
Precision: 0.7766977042089434
Recall: 0.7727713067182067
F1 Score: 0.766817962343783

LinearDiscriminantAnalysis:
Accuracy: 0.7534556617970943
Precision: 0.7830578853841017
Recall: 0.7534556617970943
F1 Score: 0.7484279963917003


C:\Users\Dodooz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")



QuadraticDiscriminantAnalysis:
Accuracy: 0.7221583859159095
Precision: 0.7471794085215752
Recall: 0.7221583859159095
F1 Score: 0.715056416890514

RidgeClassifier:
Accuracy: 0.7418441320375574
Precision: 0.7467945288932265
Recall: 0.7418441320375574
F1 Score: 0.7137356000858764

AdaBoostClassifier:
Accuracy: 0.473518389205698
Precision: 0.6863561089363764
Recall: 0.473518389205698
F1 Score: 0.5082261734146074

GradientBoostingClassifier:
Accuracy: 0.8782351233947453
Precision: 0.8876749733414683
Recall: 0.8782351233947453
F1 Score: 0.878546261311675

BaggingClassifier:
Accuracy: 0.912311321538071
Precision: 0.9190408133619888
Recall: 0.912311321538071
F1 Score: 0.9125991850152443


In [112]:
#RF-SFS-ANN
import tensorflow as tf
from tensorflow import keras

# Assuming your data is already preprocessed and loaded into X_train and y_train

# Data reshaping is likely correct: no need to reshape if labels are integers
# X_train = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1])  # Uncomment if needed (Scenario 1)
# X_test = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1])

# Define the model (assuming integer labels)
model = keras.Sequential([
  keras.Input(shape=(X_train.shape[1],)),  # Adjust input shape based on your data
  keras.layers.Dense(50, activation="tanh"),
  keras.layers.Dense(50, activation="tanh"),
  keras.layers.Dense(50, activation="tanh"),
  keras.layers.Dense(10, activation="softmax"),  # 10 neurons for 10 classes
])

# Compile the model with optimizer and loss function (assuming classification task)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


Epoch 1/10
92205/92205 ━━━━━━━━━━━━━━━━━━━━ 84s 896us/step - accuracy: 0.8030 - loss: 0.5109 - val_accuracy: 0.8387 - val_loss: 0.4014
Epoch 2/10
92205/92205 ━━━━━━━━━━━━━━━━━━━━ 85s 924us/step - accuracy: 0.8409 - loss: 0.3966 - val_accuracy: 0.8491 - val_loss: 0.3779
Epoch 3/10
92205/92205 ━━━━━━━━━━━━━━━━━━━━ 90s 972us/step - accuracy: 0.8483 - loss: 0.3767 - val_accuracy: 0.8470 - val_loss: 0.3784
Epoch 4/10
92205/92205 ━━━━━━━━━━━━━━━━━━━━ 90s 974us/step - accuracy: 0.8506 - loss: 0.3692 - val_accuracy: 0.8523 - val_loss: 0.3626
Epoch 5/10
92205/92205 ━━━━━━━━━━━━━━━━━━━━ 89s 965us/step - accuracy: 0.8520 - loss: 0.3646 - val_accuracy: 0.8543 - val_loss: 0.3607
Epoch 6/10
92205/92205 ━━━━━━━━━━━━━━━━━━━━ 89s 963us/step - accuracy: 0.8534 - loss: 0.3601 - val_accuracy: 0.8564 - val_loss: 0.3528
Epoch 7/10
92205/92205 ━━━━━━━━━━━━━━━━━━━━ 90s 970us/step - accuracy: 0.8530 - loss: 0.3598 - val_accuracy: 0.8515 - val_loss: 0.3602
Epoch 8/10
92205/92205 ━━━━━━━━━━━━━━━━━━━━ 92s 995us/s

In [117]:
#DNN
import tensorflow as tf
from tensorflow import keras

# Assuming your data is already preprocessed and loaded into X_train and y_train


# - If your data represents a single feature with multiple observations (sequences):
model = keras.Sequential([
  keras.Input(shape=(X_train.shape[1],)),  # Adjust input shape based on your data
  keras.layers.Dense(100, activation="relu"),
  keras.layers.Dense(100, activation="relu"),
  keras.layers.Dense(100, activation="relu"),
  keras.layers.Dense(10, activation="softmax"),  # 10 neurons for 10 classes
])

# Compile the model with optimizer and loss function (assuming classification task)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(X_train, y_train, epochs=100,batch_size=100, validation_data=(X_test, y_test))




Epoch 1/100
29506/29506 ━━━━━━━━━━━━━━━━━━━━ 33s 1ms/step - accuracy: 0.8014 - loss: 0.5158 - val_accuracy: 0.8467 - val_loss: 0.3834
Epoch 2/100
29506/29506 ━━━━━━━━━━━━━━━━━━━━ 32s 1ms/step - accuracy: 0.8477 - loss: 0.3785 - val_accuracy: 0.8549 - val_loss: 0.3557
Epoch 3/100
29506/29506 ━━━━━━━━━━━━━━━━━━━━ 31s 1ms/step - accuracy: 0.8535 - loss: 0.3608 - val_accuracy: 0.8579 - val_loss: 0.3481
Epoch 4/100
29506/29506 ━━━━━━━━━━━━━━━━━━━━ 31s 1ms/step - accuracy: 0.8563 - loss: 0.3522 - val_accuracy: 0.8552 - val_loss: 0.3507
Epoch 5/100
29506/29506 ━━━━━━━━━━━━━━━━━━━━ 30s 1ms/step - accuracy: 0.8585 - loss: 0.3458 - val_accuracy: 0.8595 - val_loss: 0.3425
Epoch 6/100
29506/29506 ━━━━━━━━━━━━━━━━━━━━ 31s 1ms/step - accuracy: 0.8601 - loss: 0.3411 - val_accuracy: 0.8534 - val_loss: 0.3648
Epoch 7/100
29506/29506 ━━━━━━━━━━━━━━━━━━━━ 30s 1ms/step - accuracy: 0.8619 - loss: 0.3370 - val_accuracy: 0.8606 - val_loss: 0.3356
Epoch 8/100
29506/29506 ━━━━━━━━━━━━━━━━━━━━ 31s 1ms/step - ac